<a href="https://colab.research.google.com/github/RounabhSahu/ML_Proxxmed/blob/main/U_Net_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import io
import random
import nibabel
import numpy as np
from glob import glob
import nibabel as nib
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint

from nibabel import load
import matplotlib.pyplot as plt
from keras.utils import Sequence
from IPython.display import Image, display
from skimage.exposure import rescale_intensity
from skimage.segmentation import mark_boundaries
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split
from ipywidgets import interact, interactive, fixed, interact_manual,FloatRangeSlider,FloatSlider
import ipywidgets as widgets

from google.colab import drive
from scipy.ndimage import zoom
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Set your data path
data_path = '/content/drive/MyDrive/HYPODENSITY-DATA/'

In [4]:
class BrainHypoDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_filenames, mask_filenames, batch_size, image_size):
        self.image_filenames = image_filenames
        self.mask_filenames = mask_filenames
        self.batch_size = batch_size
        self.image_size = image_size
        self.length = int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.mask_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]

        x = np.zeros((self.batch_size, *self.image_size, 1))
        y = np.zeros((self.batch_size, *self.image_size, 1))

        for i, (image_filename, mask_filename) in enumerate(zip(batch_x, batch_y)):
            image = nib.load(image_filename)
            mask = nib.load(mask_filename)
            # get the data from the image object
            image_data = image.get_fdata()
            mask_data = mask.get_fdata()
            # get random slice from the volumes
            slice_index = np.random.randint(0, image_data.shape[2] - 1)
            x[i, :, :, 0] = image_data[:, :, slice_index]
            y[i, :, :, 0] = (mask_data[:, :, slice_index] > 0).astype(np.float32)  # Assuming mask is binary

        return x, y

# Get the list of train images and masks
train_images = sorted(glob(os.path.join(data_path, '*', '*_NCCT.nii.gz')))
train_masks = sorted(glob(os.path.join(data_path, '*', '*_ROI.nii.gz')))

# Set batch size and image size
batch_size = 1
image_size = (512, 512)

# Create data generators
train_generator = BrainHypoDataGenerator(train_images[:10], train_masks[:10], batch_size, image_size)
val_generator = BrainHypoDataGenerator(train_images[10:], train_masks[10:], batch_size, image_size)

In [5]:
import tensorflow as tf

def encoder(inputs, filters, pool_size):
    conv_pool = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(inputs)
    conv_pool = tf.keras.layers.MaxPooling2D(pool_size=pool_size)(conv_pool)
    return conv_pool

def decoder(inputs, concat_input, filters, transpose_size):
    up = tf.keras.layers.Concatenate()([tf.keras.layers.Conv2DTranspose(filters, transpose_size, strides=(2, 2), padding='same')(inputs), concat_input])
    up = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(up)
    return up

def UNet(img_size=(512, 512, 1)):
    inputs = tf.keras.Input(img_size)
    print(inputs.shape)
    print()

    # Encoder
    conv_pool1 = encoder(inputs, 32, (2, 2))
    print("\t Enc. 1 ->", conv_pool1.shape)
    print()
    conv_pool2 = encoder(conv_pool1, 64, (2, 2))
    print("\t\t Enc. 2 ->", conv_pool2.shape)
    print()
    conv_pool3 = encoder(conv_pool2, 128, (2, 2))
    print("\t\t\t Enc. 3 ->", conv_pool3.shape)
    print()
    conv_pool4 = encoder(conv_pool3, 256, (2, 2))
    print("\t\t\t\t Enc. 4 ->", conv_pool4.shape)
    print()

    # Bottleneck
    bridge = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv_pool4)
    print("\t\t\t\t\t Bridge Conv ->", bridge.shape)
    print()

    # Decoder
    up6 = decoder(bridge, conv_pool3, 256, (2, 2))
    print("\t\t\t\t Dec. 4 ->", up6.shape)
    print()
    up7 = decoder(up6, conv_pool2, 128, (2, 2))
    print("\t\t\t Dec. 3 ->", up7.shape)
    print()
    up8 = decoder(up7, conv_pool1, 64, (2, 2))
    print("\t\t Dec. 2 ->", up8.shape)
    print()
    up9 = decoder(up8, inputs, 32, (2, 2))
    print("\t Dec. 1 ->", up9.shape)
    print()
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(up9)
    print(outputs.shape)
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model

# Set your input shape
input_shape = (512, 512, 1)

# Create the UNet model
model = UNet(img_size=input_shape)


(None, 512, 512, 1)

	 Enc. 1 -> (None, 256, 256, 32)

		 Enc. 2 -> (None, 128, 128, 64)

			 Enc. 3 -> (None, 64, 64, 128)

				 Enc. 4 -> (None, 32, 32, 256)

					 Bridge Conv -> (None, 32, 32, 512)

				 Dec. 4 -> (None, 64, 64, 256)

			 Dec. 3 -> (None, 128, 128, 128)

		 Dec. 2 -> (None, 256, 256, 64)

	 Dec. 1 -> (None, 512, 512, 32)

(None, 512, 512, 1)


In [ ]:

def dice_coef(y_true, y_pred, smooth=1.):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

class BrainHypoDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_filenames, mask_filenames, batch_size, image_size):
        self.image_filenames = image_filenames
        self.mask_filenames = mask_filenames
        self.batch_size = batch_size
        self.image_size = image_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
      batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
      batch_y = self.mask_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]

      x = np.zeros((self.batch_size, *self.image_size))
      y = np.zeros((self.batch_size, *self.image_size, 1))

      for i, (image_filename, mask_filename) in enumerate(zip(batch_x, batch_y)):
          image = nib.load(image_filename)
          mask = nib.load(mask_filename)
          # get the data from the image object
          image_data = image.get_fdata()
          mask_data = mask.get_fdata()
          # get random slice from the volumes
          slice_index = np.random.randint(0, image_data.shape[2] - 1)
          # Add channel dimension to image_data
          x[i, :, :, 0] = image_data[:, :, slice_index]
          # Add channel dimension to mask_data
          y[i, :, :, 0] = np.expand_dims((mask_data[:, :, slice_index] > 0).astype(np.float32), axis=-1)

      return x, y

# Get the list of train, validation, and test images and masks
train_images = sorted(glob(os.path.join(data_path, '*', '*_NCCT.nii.gz')))
train_masks = sorted(glob(os.path.join(data_path, '*', '*_ROI.nii.gz')))
val_images = sorted(glob(os.path.join(data_path, '*', '*_NCCT.nii.gz')))
val_masks = sorted(glob(os.path.join(data_path, '*', '*_ROI.nii.gz')))
test_images = sorted(glob(os.path.join(data_path, '*', '*_NCCT.nii.gz')))
test_masks = sorted(glob(os.path.join(data_path, '*', '*_ROI.nii.gz')))

# Set batch size and image size
batch_size = 1
image_size = (512, 512, 1)  # Adjusted to 3D input

# Create data generators
train_generator = BrainHypoDataGenerator(train_images, train_masks, batch_size, image_size)
val_generator = BrainHypoDataGenerator(val_images, val_masks, batch_size, image_size)
test_generator = BrainHypoDataGenerator(test_images, test_masks, batch_size, image_size)

# Build the model
def UNet(img_size=(512, 512, 1)):
    inputs = tf.keras.Input(img_size)
    print(inputs.shape)
    print()

    # Encoder
    conv_pool1 = encoder(inputs, 32, (2, 2))
    print("\t Enc. 1 ->", conv_pool1.shape)
    print()
    conv_pool2 = encoder(conv_pool1, 64, (2, 2))
    print("\t\t Enc. 2 ->", conv_pool2.shape)
    print()
    conv_pool3 = encoder(conv_pool2, 128, (2, 2))
    print("\t\t\t Enc. 3 ->", conv_pool3.shape)
    print()
    conv_pool4 = encoder(conv_pool3, 256, (2, 2))
    print("\t\t\t\t Enc. 4 ->", conv_pool4.shape)
    print()

    # Bottleneck
    bridge = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv_pool4)
    print("\t\t\t\t\t Bridge Conv ->", bridge.shape)
    print()

    # Decoder
    up6 = decoder(bridge, conv_pool3, 256, (2, 2))
    print("\t\t\t\t Dec. 4 ->", up6.shape)
    print()
    up7 = decoder(up6, conv_pool2, 128, (2, 2))
    print("\t\t\t Dec. 3 ->", up7.shape)
    print()
    up8 = decoder(up7, conv_pool1, 64, (2, 2))
    print("\t\t Dec. 2 ->", up8.shape)
    print()
    up9 = decoder(up8, inputs, 32, (2, 2))
    print("\t Dec. 1 ->", up9.shape)
    print()
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(up9)
    print(outputs.shape)
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model

# Create UNet model
model = UNet(img_size=image_size)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=[dice_coef])

# Set up model checkpoint
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=100,
                    validation_data=val_generator,
                    validation_steps=len(val_generator),
                    callbacks=[checkpoint])

(None, 512, 512, 1)

	 Enc. 1 -> (None, 256, 256, 32)

		 Enc. 2 -> (None, 128, 128, 64)

			 Enc. 3 -> (None, 64, 64, 128)

				 Enc. 4 -> (None, 32, 32, 256)

					 Bridge Conv -> (None, 32, 32, 512)

				 Dec. 4 -> (None, 64, 64, 256)

			 Dec. 3 -> (None, 128, 128, 128)

		 Dec. 2 -> (None, 256, 256, 64)

	 Dec. 1 -> (None, 512, 512, 32)

(None, 512, 512, 1)
Epoch 1/100
8/8 [==============================] - 68s 9s/step - loss: 19.8767 - dice_coef: 0.1562 - val_loss: 0.3560 - val_dice_coef: 0.0161
Epoch 2/100


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - ETA: 0s - loss: 0.1438 - dice_coef: 0.0092

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 45s 6s/step - loss: 0.1438 - dice_coef: 0.0092 - val_loss: 0.0988 - val_dice_coef: 0.0345
Epoch 3/100
8/8 [==============================] - ETA: 0s - loss: 0.0782 - dice_coef: 0.0076

...

Epoch 5/100
8/8 [==============================] - 55s 7s/step - loss: 0.0450 - dice_coef: 0.0097 - val_loss: 0.0221 - val_dice_coef: 0.0143


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 6/100
8/8 [==============================] - ETA: 0s - loss: 0.0920 - dice_coef: 0.0070

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 55s 7s/step - loss: 0.0920 - dice_coef: 0.0070 - val_loss: 0.0024 - val_dice_coef: 0.0330
Epoch 7/100
8/8 [==============================] - 56s 7s/step - loss: 0.0674 - dice_coef: 0.0045 - val_loss: 0.0420 - val_dice_coef: 0.0165
Epoch 8/100
8/8 [==============================] - 55s 7s/step - loss: 0.0038 - dice_coef: 0.0020 - val_loss: 0.0639 - val_dice_coef: 0.0068
Epoch 9/100
8/8 [==============================] - 44s 6s/step - loss: 0.0566 - dice_coef: 0.0097 - val_loss: 0.0396 - val_dice_coef: 0.0070
Epoch 10/100
8/8 [==============================] - 45s 6s/step - loss: 0.0433 - dice_coef: 0.0221 - val_loss: 0.0304 - val_dice_coef: 0.0111
Epoch 11/100
8/8 [==============================] - 56s 7s/step - loss: 0.0913 - dice_coef: 0.0334 - val_loss: 0.0699 - val_dice_coef: 0.0165
Epoch 12/100
8/8 [==============================] - 46s 6s/step - loss: 0.0074 - dice_coef: 0.0101 - val_loss: 0.0693 - val_dice_coef: 0.0248
Epoch 13/100
8/8 [====

In [ ]:
plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], color='r')
plt.plot(history.history['val_loss'])
plt.ylabel('BCE Losses')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val.'], loc='upper right')
plt.subplot(1,2,2)
plt.plot(history.history['dice_coef'], color='r')
plt.plot(history.history['val_dice_coef'])
plt.ylabel('Dice Score')
plt.xlabel('Epoch')
plt.tight_layout()
plt.show()